# 03 - Modeling (OLS Regression) & Results
In this notebook, we build a multiple linear regression model to understand how different apartment characteristics — especially construction age (Bauperiode) — influence CHF per m².

We will:

1. Load the cleaned dataset from SQLite
2. Prepare predictors and encode categorical variables
3. Fit a multiple linear regression model
4. Evaluate model diagnostics
5. Interpret coefficients and summarize insights

# 0. Setup

In [1]:
import pandas as pd
import sqlite3
import statsmodels.api as sm


# 1. Load Cleaned Data from SQLite


In [2]:

conn = sqlite3.connect("../Data/apartment_database.db")
df = pd.read_sql("SELECT * FROM rental_listings_clean", conn)
conn.close()

df.head()



,id,area_m2,rooms,rent_chf,year_built,address,canton,bauperiode,chf_per_m2,m2_per_chf
0,1,134,5.5,2530,1984,"Kesslernmattstr. 14, 8965 Berikon",Zurich,Zwischen 1981 und 1990 erbaut,18.880597,0.052964
1,2,98,3.5,2255,1989,"Im Spitzler 21, 8902 Urdorf",Zurich,Zwischen 1981 und 1990 erbaut,23.010204,0.043459
2,3,68,3.5,1945,1973,"Ferdinand Hodler-Str. 14, 8049 Zürich",Zurich,Zwischen 1971 und 1980 erbaut,28.602941,0.034961
3,4,134,5.5,5590,1906,"Weinbergstrasse 72, 8006 Zürich",Zurich,Vor 1919 erbaut,41.716418,0.023971
4,5,94,4.5,2360,1971,"Bahnhofstr. 73, 8957 Spreitenbach",Zurich,Zwischen 1971 und 1980 erbaut,25.106383,0.039831


# 2. Prepare Data for Regression 
We select the relevant variables:

Target variable (Y): chf_per_m2

Predictors (X):
- area_m2
- rooms
- year_built
- bauperiode (categorical)
- canton (categorical)

We convert categorical variables into dummy variables (one-hot encoding).

# 2.1 Create Dummies for Regression (Proper Encoding)
We use drop_first=True to avoid multicollinearity.

In [3]:
predictors = ['area_m2', 'rooms', 'year_built', 'bauperiode', 'canton']

X = pd.get_dummies(df[predictors], drop_first=True)
y = df['chf_per_m2']

X.head()


,area_m2,rooms,year_built,bauperiode_Vor 1919 erbaut,bauperiode_Zwischen 1919 und 1945 erbaut,bauperiode_Zwischen 1946 und 1960 erbaut,bauperiode_Zwischen 1961 und 1970 erbaut,bauperiode_Zwischen 1971 und 1980 erbaut,bauperiode_Zwischen 1981 und 1990 erbaut,bauperiode_Zwischen 1991 und 2000 erbaut,bauperiode_Zwischen 2001 und 2005 erbaut,bauperiode_Zwischen 2006 und 2010 erbaut,bauperiode_Zwischen 2011 und 2015 erbaut,bauperiode_Zwischen 2016 und 2020 erbaut,bauperiode_Zwischen 2021 und 2024 erbaut,canton_Luzern,canton_Zurich
0,134,5.5,1984,False,False,False,False,False,True,False,False,False,False,False,False,False,True
1,98,3.5,1989,False,False,False,False,False,True,False,False,False,False,False,False,False,True
2,68,3.5,1973,False,False,False,False,True,False,False,False,False,False,False,False,False,True
3,134,5.5,1906,True,False,False,False,False,False,False,False,False,False,False,False,False,True
4,94,4.5,1971,False,False,False,False,True,False,False,False,False,False,False,False,False,True


Baseline categories (automatically chosen):

- bauperiode: Vor 1919 gebaut
- canton: Bern

# 2.2 (Optional) Full Dummies for Descriptive Comparison
All 3 cantons visible for tables/plots.

In [4]:
canton_full = pd.get_dummies(df['canton'], drop_first=False)
canton_full.head()


,Bern,Luzern,Zurich
0,False,False,True
1,False,False,True
2,False,False,True
3,False,False,True
4,False,False,True


## 3. Fit Multiple Linear Regression (OLS)
We use Statsmodels OLS because it provides a detailed summary table with:
- coefficients
- p-values
- confidence intervals
- R² and adjusted R²

In [6]:
# Ensure all dummy variables are numeric (0/1)
X = X.apply(pd.to_numeric, errors='coerce')

# Ensure Y is numeric
y = pd.to_numeric(y, errors='coerce')


In [7]:
X = X.astype(float)
# 1. Create dummy matrix
X = pd.get_dummies(df[predictors], drop_first=True)

# 2. Convert everything to numeric
X = X.apply(pd.to_numeric, errors='coerce')

# 3. Convert booleans to floating point
X = X.astype(float)

# 4. Target
y = pd.to_numeric(df['chf_per_m2'], errors='coerce')


In [8]:
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             chf_per_m2   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     6.643
Date:                Mon, 01 Dec 2025   Prob (F-statistic):           2.36e-10
Time:                        13:47:59   Log-Likelihood:                -397.08
No. Observations:                 120   AIC:                             830.2
Df Residuals:                     102   BIC:                             880.3
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
const                                     -111.9630    510.412     -0.219      0.827   -1124.363     900.437
area_m2                                     -0.0712      0.038     -1.855      0.066      -0.147       0.005
rooms                                       -0.8053      1.167     -0.690      0.492      -3.119       1.509
year_built                                   0.0722      0.252      0.286      0.775      -0.428       0.572
bauperiode_Vor 1919 erbaut                  13.6504     31.400      0.435      0.665     -48.631      75.932
bauperiode_Zwischen 1919 und 1945 erbaut    11.2468     21.667      0.519      0.605     -31.730      54.224
bauperiode_Zwischen 1946 und 1960 erbaut     3.6609     17.380      0.211      0.834     -30.812      38.134
bauperiode_Zwischen 1961 und 1970 erbaut    -0.3523     15.317     -0.023      0.982     -30.733      30.029
bauperiode_Zwischen 1971 und 1980 erbaut    -3.8245     13.228     -0.289      0.773     -30.062      22.413
bauperiode_Zwischen 1981 und 1990 erbaut    -5.2300     10.926     -0.479      0.633     -26.902      16.442
bauperiode_Zwischen 1991 und 2000 erbaut    -3.4273      8.374     -0.409      0.683     -20.037      13.183
bauperiode_Zwischen 2001 und 2005 erbaut    -3.1528      7.015     -0.449      0.654     -17.068      10.762
bauperiode_Zwischen 2006 und 2010 erbaut    -2.8252      5.150     -0.549      0.584     -13.040       7.390
bauperiode_Zwischen 2011 und 2015 erbaut     0.6774      4.079      0.166      0.868      -7.412       8.767
bauperiode_Zwischen 2016 und 2020 erbaut    -0.5199      3.890     -0.134      0.894      -8.236       7.196
bauperiode_Zwischen 2021 und 2024 erbaut     6.7349      3.065      2.198      0.030       0.656      12.814
canton_Luzern                                7.6909      1.739      4.423      0.000       4.242      11.139
canton_Zurich                               11.4320      1.764      6.480      0.000       7.933      14.931
==============================================================================
Omnibus:                       31.288   Durbin-Watson:                   1.698
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.618
Skew:                           1.092   Prob(JB):                     4.17e-14
Kurtosis:                       5.748   Cond. No.                     1.56e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.56e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""